# Playground CHATGPT
Author : Patricio Ortiz

The main idea of this notebook is to handle the data of some experiments, and be able to work with them in order to get a CHATGPT response with the diagnosis and the actions proposed to the client.

## Libraries

In [3]:
import pandas as pd
import numpy as np
import openai
import os

from aux_functions import *

openai.api_key = os.getenv("OPENAI_API_KEY")

## Data

In [4]:
df_val = pd.read_csv('datos procesados/valores.csv')
dfi = pd.read_csv('datos procesados/componentes.csv', low_memory=False)
df2 = pd.read_csv('datos procesados/valores_raw.csv', low_memory=False)

## Playground

In [20]:
l = []
ids = dfi.iloc[37850:37855,0].values.tolist()

for i in ids:
  c = generate_comment(i)
  t1 = c[0]
  t2 = c[1]
  t = (t1, t2)
  
  l.append(t)

In [30]:
i = 5


print(ids[i])
print('\n')
t3 = df2[df2.id_sample == ids[i]]
display(t3)
print('\n')
t4 = dfi[dfi.id_sample == ids[i]][['name_component','name_component_brand','name_component_model','name_component_type','name_machine','name_machine_type','name_machine_brand','name_machine_model']]
display(t4.dropna(axis=1))
print('\n')
k = l[i][1]
print(l[i][1])
print('\n')
print(l[i][0])
print('\n')


IndexError: list index out of range

In [23]:
t4.to_string(index=False)

'name_component name_component_brand name_component_model name_component_type       name_machine  name_machine_type name_machine_brand name_machine_model\n  MOTOR DIESEL                  NaN                  NaN        MOTOR DIESEL PERFORADORA D - 06 FLOTA PERFORADORAS        ATLAS COPCO                NaN'